In [1]:
# workaround for incredibly stupid vscode on snap bug
from os import environ
krb5ccname = environ["KRB5CCNAME"]
if "hostfs" in krb5ccname:
    krb5ccname = krb5ccname.replace("/var/lib/snapd/hostfs", "")
    environ["KRB5CCNAME"] = krb5ccname

In [2]:
import ROOT
from analysis_framework import Dataset
from OptimalObservableHelper import OptimalObservableHelper
from AltSetupHandler import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9238bd0


In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
# write_outputs = False
write_outputs = True
# plot_dir_postfix = "-new-cuts"
dataset_path = "data/datasets/reweighted/test.json"
# output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/test"
# output_meta_path = "data/datasets/reweighted"
# output_meta = f"{output_meta_path}/test.json"
# checked_output_meta = f"{output_meta_path}/checked-test.json"
# output_collections = [
#     "true_lep_lvec", "true_nu_lvec", "true_quark1_lvec", "true_quark2_lvec",
#     "iso_lep_lvec", "nu_lvec", "R2Jet_sel1_lvec", "R2Jet_sel2_lvec",
#     ]
# true lvecs do not exist in every df so cannot be explicitly requested...
# urgh but empty snapshots are also not allowed
# output_collections = r"(true_\w+_lvec)|(iso_lep_lvec)|(nu_lvec)|(R2Jet_sel1_lvec)|(R2Jet_sel2_lvec)|(\w*sqme\w*)|(weight\w*)"
# plot_dir = f"plots/pre-selection/test{plot_dir_postfix}"
if prod:
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs-min-aa-min-higgs.json"
    # dataset_path = "data/datasets/miniDSTs/min-higgs.json"
    dataset_path = "data/datasets/reweighted/signal-only.json"
    # output_path = "root://eospublic.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/min-higgs-d"
    # output_path = "root://eosuser.cern.ch//eos/user/l/lreichen/TGC2/data/reweighted/signal-only"
    # output_meta_path = "data/datasets/reweighted"
    # output_meta = f"{output_meta_path}/signal-only.json"
    # checked_output_meta = f"{output_meta_path}/checked-signal-only.json"
    # plot_dir = "plots/pre-selection/full"
    # plot_dir = f"plots/pre-selection/min-higgs{plot_dir_postfix}"


In [4]:
ROOT.EnableImplicitMT(n_threads)
# environ["OMP_NUM_THREADS"] = "6"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = OptimalObservableHelper(dataset)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x823b450


In [7]:
analysis.init_categories()
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]

True


In [8]:
oo_configs = [
    "g1z_pos_1em08",
    "ka_pos_1em08",
    "la_pos_1em08",
    ]
# analysis.define_optimal_observables_truth(oo_configs, truth_categories=signal_category)
# analysis.define_optimal_observables_truth_averaged(oo_configs, truth_categories=signal_category)
# analysis.define_optimal_observables_reco(oo_configs)
# oo_names = [f"O_{c}" for c in oo_configs]
oo_names = {
    "reco_oo": analysis.define_optimal_observables("O", ["reco_sqme", "wj_reco_sqme"], oo_configs, categories=signal_category),
    "reco_jm_oo": analysis.define_optimal_observables("jm_O", ["reco_sqme"], oo_configs, categories=signal_category),
    "clean_reco_oo": analysis.define_optimal_observables("clean_O", ["clean_reco_sqme", "wj_clean_reco_sqme"], oo_configs, categories=signal_category),
    "clean_reco_jm_oo": analysis.define_optimal_observables("clean_jm_O", ["clean_reco_sqme"], oo_configs, categories=signal_category),
    "cheat_clean_reco_oo": analysis.define_optimal_observables("cheat_clean_O", ["cheat_clean_reco_sqme", "wj_cheat_clean_reco_sqme"], oo_configs, categories=signal_category),
    "cheat_clean_reco_jm_oo": analysis.define_optimal_observables("cheat_clean_jm_O", ["cheat_clean_reco_sqme"], oo_configs, categories=signal_category),
    "mc_oo": analysis.define_optimal_observables("mc_O", ["mc_sqme"], oo_configs, categories=signal_category),
    "nomb_mc_oo": analysis.define_optimal_observables("nomb_mc_O", ["nomb_mc_sqme"], oo_configs, categories=signal_category),
    "nomb_mc_rlep_oo": analysis.define_optimal_observables("nomb_mc_rlep_O", ["nomb_mc_rlep_sqme"], oo_configs, categories=signal_category),
    "av_mc_oo": analysis.define_optimal_observables("av_mc_O", ["mc_sqme", "wj_mc_sqme"], oo_configs, categories=signal_category),
    "av_nomb_mc_oo": analysis.define_optimal_observables("av_nomb_mc_O", ["nomb_mc_sqme", "wj_nomb_mc_sqme"], oo_configs, categories=signal_category),
    "av_nomb_mc_rlep_oo": analysis.define_optimal_observables("av_nomb_mc_rlep_O", ["nomb_mc_rlep_sqme", "wj_nomb_mc_rlep_sqme"], oo_configs, categories=signal_category),
}

In [9]:
for names in oo_names.values():
    for name in names:
        # create dummy oo where they don't exist yet
        analysis.define_only_on(["4f_sl_bkg"], name, "0.")
# TODO: re-consider what is correct here...
analysis.add_filter("&&".join([f"std::isfinite({oo})" for oo_name in oo_names.values() for oo in oo_name]), "finite OO")
# analysis.add_filter("&&".join([f"abs({oo}) <= 5" for oo in oo_names["reco_oo"]]), "abs(OO) <= 5")
# analysis.add_filter("(iso_lep_lvec + nu_lvec).M() > 0.", "physical nu")
# analysis.add_filter("(iso_lep_lvec + clean_nu_lvec).M() > 0.", "physical clean_nu")
# analysis.add_filter("nu_lvec.E() > 0.", "physical nu")
# analysis.add_filter("clean_nu_lvec.E() > 0.", "physical clean nu")
analysis.book_reports()

In [10]:
analysis.book_histogram_1D("mc_O_g1z_pos_1em08", "mc_O_g1z_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_ka_pos_1em08", "mc_O_ka_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)
analysis.book_histogram_1D("mc_O_la_pos_1em08", "mc_O_la_pos_1em08", ("", "", 250, -2.5, 2.5), categories=signal_category)

In [11]:
alt_setup_handler = AltSetupHandler("""
{
  "SM": {
    "g1z": 1.0,
    "ka": 1.0,
    "la": 0.0
  },
"variations": [
    0.001,
    -0.001,
    1e-04,
    -1e-04,
    1e-08
  ]
}
""", mirror=False, combinations=False)
alt_config_names = list(alt_setup_handler.get_alt_setup().keys())

In [12]:
weight_names = analysis.book_weight_sums(["nominal"] + alt_config_names, categories=signal_category)

In [13]:
for names in oo_names.values():
    analysis.define_weighted_oo(names, weight_names, categories=signal_category)
    analysis.book_oo_sums(names, weight_names, categories=signal_category)
    analysis.book_oo_matrix(names, categories=signal_category)

In [14]:
%%time
analysis.run()

CPU times: user 2min 12s, sys: 1.63 s, total: 2min 14s
Wall time: 27.4 s


In [15]:
analysis.print_reports()

         4f_sw_sl_signal               4f_sl_bkg
        10244400 (1e+04)           33239 (6e+02) All
        10236314 (1e+04)           33239 (6e+02) finite OO
                    1.00                    1.00 efficiency



In [16]:
# calculate means etc.
# oo_means = analysis.calc_oo_means(oo_names, weight_names, categories=signal_category)

In [17]:
print(alt_config_names)

['g1z_pos_1em03', 'ka_pos_1em03', 'la_pos_1em03', 'g1z_neg_1em03', 'ka_neg_1em03', 'la_neg_1em03', 'g1z_pos_1em04', 'ka_pos_1em04', 'la_pos_1em04', 'g1z_neg_1em04', 'ka_neg_1em04', 'la_neg_1em04', 'g1z_pos_1em08', 'ka_pos_1em08', 'la_pos_1em08']


In [18]:
# oo_graphs = analysis.make_slope_graphs(oo_names, ["g1z", "ka", "la"], alt_config_names, oo_means)

In [19]:
# canvases = {}
# for k, g in oo_graphs.items():
#     c = ROOT.TCanvas()
#     g.Draw("alp")
#     c.Draw()
#     canvases[k] = c

In [20]:
if write_outputs:
    for name, names in oo_names.items():
        print(name)
        analysis.print_fit_input(names, weight_names, e_pol=0., p_pol=0., categories=signal_category, dir="fit-configs/signal-only", name=name)

reco_oo
#evt/ab_inv, means, slopes, cov
2047262.771679877
[0.009050337751655086, 0.02986366191236436, 0.07941814171753178]
[[56.122630789087246, 11.07237941246507, -35.34526906037747], [2.0228391370594236, 7.200556792461437, -0.6056150163560191], [-5.144198311514213, -0.5953383330757157, 9.551647487744916]]
[[0.7188752700091822, 0.1545973079123668, -0.3673602443525501], [0.1545973079123668, 0.45951923657092947, -0.009535052450613564], [-0.3673602443525501, -0.009535052450613564, 1.497112484751196]]

reco_jm_oo
#evt/ab_inv, means, slopes, cov
2047262.771679877
[0.03426759426688541, 0.015816566507402534, 0.0593572042825751]
[[29.928850957383453, -2.190463374087776, -17.38482652950051], [-6.22812007868442, 22.118290368293408, 0.8575828968437179], [-10.429072347800265, -0.614732178907253, 29.76956128440743]]
[[1.843129963804532, -0.021205946634546016, -0.8167863220401246], [-0.021205946634546016, 0.82812993035282, -0.11679953728680001], [-0.8167863220401246, -0.11679953728680001, 3.5047562

cheat_clean_reco_oo
#evt/ab_inv, means, slopes, cov
2047262.771679877
[0.020199000878318503, 0.08944565825042262, 0.12401454044702803]
[[27.271726567059353, 4.708209766974348, -17.737021776064392], [0.5072885637248303, 2.399463164417753, -0.16905907007045776], [-3.5413919453643006, -0.6697682675660231, 6.490692580098767]]
[[0.6432501425982261, 0.10295928446266547, -0.3770357070217203], [0.10295928446266547, 0.37982418981067334, 0.009411421585742218], [-0.3770357070217203, 0.009411421585742218, 1.1070707868908578]]

cheat_clean_reco_jm_oo
#evt/ab_inv, means, slopes, cov
2047262.771679877
[0.027373226199887257, 0.08214875975211186, 0.12146134069573557]
[[38.31060647542124, -2.099999696244751, -21.99535179958082], [-1.3279031410923332, 4.166258708071064, 0.03429225265721244], [-5.561037428699802, -0.4909117501075932, 15.284000709601555]]
[[1.4680939105378537, -0.0348527074212069, -0.7124866953497548], [-0.0348527074212069, 0.6252464705580689, 0.004387170857465357], [-0.7124866953497548, 0